In [3]:
import pandas as pd
import duckdb
import os

print("Pandas version:", pd.__version__)
print("DuckDB version:", duckdb.__version__)

Pandas version: 2.3.3
DuckDB version: 1.4.3


# Dataset Occurrence u observaciones

In [3]:
import duckdb
import pandas as pd

urlo = 'https://www.dropbox.com/scl/fi/i9sshclfbojl8az397ce0/Occurrence.txt?rlkey=daw5f2p78yn1r5bt67vdyk1yv&st=0yc5vcl1&dl=1'

con = duckdb.connect(database=':memory:')

# Traer los primeros 5 registros a Pandas usando la variable
df = con.execute(f"""
    SELECT *
    FROM read_csv_auto('{urlo}')
    LIMIT 5
""").fetchdf()  # o .df() también funciona

df.head()

,id,occurrenceID,catalogNumber,basisOfRecord,collectionCode,dynamicProperties,otherCatalogNumbers,genus,specificEpithet,infraspecificEpithet,...,verbatimEventDate,fieldNotes,behavior,sex,lifeStage,preparations,references,Associated Taxa,rightsHolder,license
0,9@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC9,HumanObservation,Wildlife sounds - Birds,None,None,Synallaxis,azarae,media,...,12-08-2002,two birds trip:http://www.cs.bris.ac.uk/home/p...,song,None,None,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,None,Bob Planqué,CC BY-NC
1,9@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC9,HumanObservation,Wildlife sounds - Birds,None,None,Synallaxis,azarae,media,...,12-08-2002,two birds trip:http://www.cs.bris.ac.uk/home/p...,song,None,None,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,None,Bob Planqué,CC BY-NC
2,99@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC99,HumanObservation,Wildlife sounds - Birds,None,None,Turdus,hauxwelli,None,...,01-10-2003,trip:http://www.cs.bris.ac.uk/home/planque/Peru/,song,None,None,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,None,Bob Planqué,CC BY-NC
3,99@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC99,HumanObservation,Wildlife sounds - Birds,None,None,Turdus,hauxwelli,None,...,01-10-2003,trip:http://www.cs.bris.ac.uk/home/planque/Peru/,song,None,None,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,None,Bob Planqué,CC BY-NC
4,9999@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC9999,HumanObservation,Wildlife sounds - Birds,None,None,Myrmothera,campanisona,signata,...,04-03-1998,None,song,None,None,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,has background sounds: Selenidera reinwardtii,Allen T. Chartier,CC BY-NC


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     5 non-null      object        
 1   occurrenceID           5 non-null      object        
 2   catalogNumber          5 non-null      object        
 3   basisOfRecord          5 non-null      object        
 4   collectionCode         5 non-null      object        
 5   dynamicProperties      0 non-null      object        
 6   otherCatalogNumbers    0 non-null      object        
 7   genus                  5 non-null      object        
 8   specificEpithet        5 non-null      object        
 9   infraspecificEpithet   3 non-null      object        
 10  scientificName         5 non-null      object        
 11  taxonRank              5 non-null      object        
 12  kingdom                5 non-null      object        
 13  family   

In [5]:
import duckdb

urlo = 'https://www.dropbox.com/scl/fi/i9sshclfbojl8az397ce0/Occurrence.txt?rlkey=daw5f2p78yn1r5bt67vdyk1yv&st=0yc5vcl1&dl=1'
con = duckdb.connect(database=':memory:')

# Contar ocurrencias por país y ordenar de mayor a menor
df_country_counts = con.execute(f"""
    SELECT country, COUNT(*) AS num_records
    FROM read_csv_auto('{urlo}')
    GROUP BY country
    ORDER BY num_records DESC
""").fetchdf()

# Mostrar los primeros países con más registros
print(df_country_counts.head(20))

           country  num_records
0    United States       136821
1           Brazil       129488
2   United Kingdom       127166
3           France       113395
4         Colombia        68952
5          Ecuador        63606
6            Spain        62638
7          Germany        60480
8           Sweden        47175
9           Mexico        43797
10          Poland        39300
11     Netherlands        35448
12           China        34261
13           India        33301
14            Peru        31302
15       Australia        30171
16    South Africa        28304
17       Argentina        26280
18       Indonesia        22773
19        Malaysia        22524


In [6]:
# Aproximación geográfica de Europa usando las columnas correctas
min_lat, max_lat = 35.0, 71.0
min_lon, max_lon = -10.0, 40.0

urlo = 'https://www.dropbox.com/scl/fi/i9sshclfbojl8az397ce0/Occurrence.txt?rlkey=daw5f2p78yn1r5bt67vdyk1yv&st=0yc5vcl1&dl=1'
df_europe_geo = con.execute(f"""
    SELECT *
    FROM read_csv_auto('{urlo}')
    WHERE latitudeDecimal BETWEEN {min_lat} AND {max_lat}
      AND longitudeDecimal BETWEEN {min_lon} AND {max_lon}
    LIMIT 1000
""").fetchdf()

df_europe_geo.head()

print("Número de filas:", len(df_europe_geo))
print("Número de columnas:", df_europe_geo.shape[1])

Número de filas: 1000
Número de columnas: 37


In [7]:
# Aproximación geográfica de Europa sin límite
min_lat, max_lat = 35.0, 71.0
min_lon, max_lon = -10.0, 40.0

urlo = 'https://www.dropbox.com/scl/fi/i9sshclfbojl8az397ce0/Occurrence.txt?rlkey=daw5f2p78yn1r5bt67vdyk1yv&st=0yc5vcl1&dl=1'
df_europe_geo = con.execute(f"""
    SELECT *
    FROM read_csv_auto('{urlo}')
    WHERE latitudeDecimal BETWEEN {min_lat} AND {max_lat}
      AND longitudeDecimal BETWEEN {min_lon} AND {max_lon}
""").fetchdf()

print("Número de filas:", len(df_europe_geo))
print("Número de columnas:", df_europe_geo.shape[1])

Número de filas: 627360
Número de columnas: 37


In [10]:
import duckdb

# URL del CSV en Dropbox
urlo = 'https://www.dropbox.com/scl/fi/i9sshclfbojl8az397ce0/Occurrence.txt?rlkey=daw5f2p78yn1r5bt67vdyk1yv&st=0yc5vcl1&dl=1'

# Conexión en memoria
con = duckdb.connect()

# Ejecutar consulta SQL con f-string para interpolar la variable
result = con.execute(f"""
    SELECT *
    FROM read_csv_auto('{urlo}')
    WHERE country = 'Spain'
""").df()

# Mostrar resultados
result.head()

,id,occurrenceID,catalogNumber,basisOfRecord,collectionCode,dynamicProperties,otherCatalogNumbers,genus,specificEpithet,infraspecificEpithet,...,verbatimEventDate,fieldNotes,behavior,sex,lifeStage,preparations,references,Associated Taxa,rightsHolder,license
0,999933@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC999933,MachineObservation,Wildlife sounds - Birds,None,None,Rallus,aquaticus,None,...,2025-05-20,animal seen:no; playback used:no,nocturnal flight call,None,None,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,None,Jorge Rodal,CC BY-NC
1,999870@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC999870,HumanObservation,Wildlife sounds - Birds,"{""recordingDevice"":""Zoom H5"",""microphone"":""Tel...",None,Cettia,cetti,None,...,2025-05-17,Singing from a patch of brambles.; animal seen...,song,undetermined,uncertain,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,has background sounds: Cuculus canorus|Oriolus...,João Tomás,CC BY-NC
2,999857@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC999857,HumanObservation,Wildlife sounds - Birds,"{""recordingDevice"":""Zoom h1n"",""microphone"":""2x...",None,Curruca,iberiae,None,...,2025-04-20,animal seen:yes; playback used:no,"song, call",male,adult,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,None,Jorge Valella Robledo,CC BY-NC
3,999856@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC999856,HumanObservation,Wildlife sounds - Birds,"{""recordingDevice"":""Zoom h1n"",""microphone"":""2x...",None,Curruca,undata,None,...,2025-04-27,animal seen:yes; playback used:no,song,male,None,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,None,Jorge Valella Robledo,CC BY-NC
4,999830@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC999830,MachineObservation,Wildlife sounds - Birds,"{""recordingDevice"":""PARABOLIC DISH""}",None,Passer,domesticus,None,...,2025-05-19,animal seen:no; playback used:no,call,undetermined,uncertain,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,None,Sergi Carreras,CC BY-NC


# Datos de dataset Multimedia

In [13]:

con = duckdb.connect(database=':memory:')

urlm = 'https://www.dropbox.com/scl/fi/z8lpwp3qnvz0r8lsc5qi1/Multimedia.txt?rlkey=yk05peoav2vw5ojbkqsebx9n7&st=q63xenjk&dl=1'

# Traer los primeros 5 registros a Pandas
dfm = con.execute(f"""
    SELECT *
    FROM read_csv_auto('{urlm}')
    LIMIT 5
""").fetchdf()

dfm.head()



,CoreId,associatedObservationReference,Identifier,type,Rating,rightsHolder,creator,accessURI,format,variantLiteral,description,caption,resourceCreationTechnique,captureDevice,physicalSetting,license
0,9@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,https://xeno-canto.org/sounds/uploaded/OH38YHK...,StillImage,<NA>,Stichting Xeno-canto voor Natuurgeluiden,Stichting Xeno-canto voor Natuurgeluiden,https://xeno-canto.org/sounds/uploaded/OH38YHK...,image/png,ac:MediumQuality,None,Oscillogram of the first ten seconds of the so...,None,None,None,CC BY-NC-SA 3.0
1,9@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,https://xeno-canto.org/sounds/uploaded/OH38YHK...,StillImage,<NA>,Stichting Xeno-canto voor Natuurgeluiden,Stichting Xeno-canto voor Natuurgeluiden,https://xeno-canto.org/sounds/uploaded/OH38YHK...,image/png,ac:MediumQuality,None,Oscillogram of the first ten seconds of the so...,None,None,None,CC BY-NC-SA 3.0
2,9@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,https://xeno-canto.org/sounds/uploaded/OH38YHK...,StillImage,<NA>,Stichting Xeno-canto voor Natuurgeluiden,Stichting Xeno-canto voor Natuurgeluiden,https://xeno-canto.org/sounds/uploaded/OH38YHK...,image/png,ac:MediumQuality,None,Spectrogram of the first ten seconds of the so...,None,None,None,CC BY-NC-SA 3.0
3,9@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,https://xeno-canto.org/sounds/uploaded/OH38YHK...,StillImage,<NA>,Stichting Xeno-canto voor Natuurgeluiden,Stichting Xeno-canto voor Natuurgeluiden,https://xeno-canto.org/sounds/uploaded/OH38YHK...,image/png,ac:MediumQuality,None,Spectrogram of the first ten seconds of the so...,None,None,None,CC BY-NC-SA 3.0
4,9@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,https://xeno-canto.org/sounds/uploaded/OH38YHK...,Sound,5,Bob Planqué,Bob Planqué,https://xeno-canto.org/sounds/uploaded/OH38YHK...,audio/mp3,ac:BestQuality,17 s,None,automatic recording: no; bitrate: 64000 bps; b...,None,Natural,CC BY-NC-SA 3.0


In [ ]:
dfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   CoreId                          5 non-null      object
 1   associatedObservationReference  5 non-null      object
 2   Identifier                      5 non-null      object
 3   type                            5 non-null      object
 4   Rating                          1 non-null      Int64 
 5   rightsHolder                    5 non-null      object
 6   creator                         5 non-null      object
 7   accessURI                       5 non-null      object
 8   format                          5 non-null      object
 9   variantLiteral                  5 non-null      object
 10  description                     1 non-null      object
 11  caption                         4 non-null      object
 12  resourceCreationTechnique       1 non-null      object

# Union de los datasets Occurrence.txt y Multimedia.txt con ID=CoreId

In [4]:

# 🔹 Conexión en memoria
con = duckdb.connect(database=':memory:')

# 🔹 URLs de los archivos en Dropbox
urlo = "https://www.dropbox.com/scl/fi/i9sshclfbojl8az397ce0/Occurrence.txt?rlkey=daw5f2p78yn1r5bt67vdyk1yv&st=0yc5vcl1&dl=1"
urlm = "https://www.dropbox.com/scl/fi/z8lpwp3qnvz0r8lsc5qi1/Multimedia.txt?rlkey=yk05peoav2vw5ojbkqsebx9n7&st=q63xenjk&dl=1"

# 🔹 Crear tabla Occurrence filtrada solo para España y Alemania
con.execute(f"""
CREATE TABLE occurrence_es AS
SELECT *
FROM read_csv_auto('{urlo}', header=True)
WHERE country IN ('Spain', 'Germany')
""")

# 🔹 Crear tabla Multimedia
con.execute(f"""
CREATE TABLE multimedia AS
SELECT *
FROM read_csv_auto('{urlm}', header=True)
""")

# 🔹 Hacer JOIN usando Occurrence.id = Multimedia.CoreId
query = """
SELECT 
    o.*, 
    m.Identifier, 
    m.type, 
    m.format, 
    m.accessURI
FROM occurrence_es o
LEFT JOIN multimedia m
    ON o.id = m.CoreId
"""

# 🔹 Ejecutar consulta y obtener DataFrame
# df_joined = con.execute(query).fetchdf()

# 🔹 Mostrar las primeras filas
# print(df_joined.head())

*Limpieza de duplicados y agrupacion Dataset filtrado con datos de España y Alemania

In [5]:
import duckdb

# URL del archivo en Dropbox
url = 'https://www.dropbox.com/scl/fi/bgb8rrylp65egjs56m2jc/df_joined.csv?rlkey=0ntccu09bu9v7qujj5prq8nqz&st=tihqh9le&dl=1'

# Conexión en memoria
con = duckdb.connect(database=':memory:')

# Leer CSV directamente desde la URL
df = con.execute(f"""
    SELECT *
    FROM read_csv_auto('{url}')
""").fetchdf()

# Mostrar las primeras filas
df.head()

,id,occurrenceID,catalogNumber,basisOfRecord,collectionCode,dynamicProperties,otherCatalogNumbers,genus,specificEpithet,infraspecificEpithet,...,lifeStage,preparations,references,Associated Taxa,rightsHolder,license,Identifier,type,format,accessURI
0,839019@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC839019,HumanObservation,Wildlife sounds - Birds,"{""recordingDevice"":""MIXPRE 6"",""microphone"":""Te...",None,Hieraaetus,pennatus,None,...,uncertain,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,has background sounds: Phylloscopus collybita,Jordi Calvet,CC BY-NC,https://xeno-canto.org/sounds/uploaded/PNYKOPB...,StillImage,image/png,https://xeno-canto.org/sounds/uploaded/PNYKOPB...
1,839019@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC839019,HumanObservation,Wildlife sounds - Birds,"{""recordingDevice"":""MIXPRE 6"",""microphone"":""Te...",None,Hieraaetus,pennatus,None,...,uncertain,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,has background sounds: Phylloscopus collybita,Jordi Calvet,CC BY-NC,https://xeno-canto.org/sounds/uploaded/PNYKOPB...,StillImage,image/png,https://xeno-canto.org/sounds/uploaded/PNYKOPB...
2,839019@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC839019,HumanObservation,Wildlife sounds - Birds,"{""recordingDevice"":""MIXPRE 6"",""microphone"":""Te...",None,Hieraaetus,pennatus,None,...,uncertain,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,has background sounds: Phylloscopus collybita,Jordi Calvet,CC BY-NC,https://xeno-canto.org/sounds/uploaded/PNYKOPB...,StillImage,image/png,https://xeno-canto.org/sounds/uploaded/PNYKOPB...
3,839019@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC839019,HumanObservation,Wildlife sounds - Birds,"{""recordingDevice"":""MIXPRE 6"",""microphone"":""Te...",None,Hieraaetus,pennatus,None,...,uncertain,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,has background sounds: Phylloscopus collybita,Jordi Calvet,CC BY-NC,https://xeno-canto.org/sounds/uploaded/PNYKOPB...,StillImage,image/png,https://xeno-canto.org/sounds/uploaded/PNYKOPB...
4,839019@XC,https://data.biodiversitydata.nl/xeno-canto/ob...,XC839019,HumanObservation,Wildlife sounds - Birds,"{""recordingDevice"":""MIXPRE 6"",""microphone"":""Te...",None,Hieraaetus,pennatus,None,...,uncertain,field recording,https://data.biodiversitydata.nl/xeno-canto/ob...,has background sounds: Phylloscopus collybita,Jordi Calvet,CC BY-NC,https://xeno-canto.org/sounds/uploaded/PNYKOPB...,Sound,audio/mp3,https://xeno-canto.org/sounds/uploaded/PNYKOPB...


In [6]:
df.shape

(725663, 41)

In [8]:
#limpiar dataset de duplicados 


# 1️⃣ Ver duplicados exactos
duplicados = df[df.duplicated()]
print(f"Número de filas duplicadas exactas: {len(duplicados)}")

# 2️⃣ Ver duplicados por 'id' (varias imágenes por ocurrencia)
duplicados_id = df[df.duplicated(subset=['id'], keep=False)]
print(f"Número de IDs con varias filas: {duplicados_id['id'].nunique()}")

# 3️⃣ Eliminar duplicados exactos
df_sin_duplicados = df.drop_duplicates()
print(f"Tamaño después de eliminar duplicados exactos: {df_sin_duplicados.shape}")

# 4️⃣ Agrupar múltiples imágenes por ocurrencia 
df_agrupado = df_sin_duplicados.groupby('id').agg({
    'Identifier': list,
    'type': list,
    'format': list,
    'accessURI': list,
    'country': 'first',  # mantener país
    # puedes agregar otras columnas de Occurrence que quieras conservar
}).reset_index()

print(f"Tamaño después de agrupar por ID: {df_agrupado.shape}")

# 5️⃣ Guardar DataFrame limpio
#df_agrupado.to_csv('df_joined_limpio.csv', index=False)


#print("¡DataFrame limpio guardado en CSV!")


Número de filas duplicadas exactas: 536234
Número de IDs con varias filas: 62692
Tamaño después de eliminar duplicados exactos: (189429, 41)
Tamaño después de agrupar por ID: (63475, 6)
